In [1]:
import re

article_re = re.compile(r"المادة\s*([0-9]+)")

def extract_article_number(text: str):
    m = article_re.search(text)
    return m.group(1) if m else None

In [2]:
import json

# load chunks (same file you used before)
with open(r"C:\Users\PC\OneDrive\Desktop\chatbot\data\processed\penal_code_chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

metadatas = []
last_seen_article = None

for ch in chunks:
    art = extract_article_number(ch)
    if art is not None:
        last_seen_article = art
        metadatas.append({"article": art, "is_continuation": False})
    else:
        metadatas.append({"article": last_seen_article, "is_continuation": True})


In [3]:
import chromadb
from chromadb.utils import embedding_functions
import numpy as np

# load your upgraded embeddings (e5-large) you saved earlier
embeddings = np.load(r"C:\Users\PC\OneDrive\Desktop\chatbot\data\embeddings\upgrade\penal_code_e5_embeddings.npy")

# connect with embedding function so queries also use e5-large
embed_func = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="intfloat/multilingual-e5-large"
)
chroma_client = chromadb.PersistentClient(
    path=r"C:\Users\PC\OneDrive\Desktop\chatbot\data\chroma_db_upgrade"
)

# nuke & recreate to be safe
try:
    chroma_client.delete_collection("penal_code_upgrade")
except Exception:
    pass

collection = chroma_client.create_collection(
    name="penal_code_upgrade",
    embedding_function=embed_func
)

ids = [f"chunk_{i}" for i in range(len(chunks))]

collection.add(
    ids=ids,
    documents=chunks,
    embeddings=embeddings.tolist(),
    metadatas=metadatas
)

print(f"✅ Rebuilt with metadata: {len(chunks)} chunks")


✅ Rebuilt with metadata: 321 chunks


In [4]:
def retrieve_with_meta(query: str, top_k: int = 3):
    res = collection.query(
        query_texts=[query],
        n_results=top_k,
        include=["documents", "distances", "metadatas"]  # 👈 added metadatas
    )
    docs  = res["documents"][0]
    dists = res["distances"][0]
    metas = res["metadatas"][0]
    sims  = [1 - d for d in dists]
    return list(zip(sims, metas, docs))

# quick test
query = "ما هي العقوبة على السرقة؟"
hits = retrieve_with_meta(query, 3)
for i, (sim, meta, doc) in enumerate(hits, 1):
    print(f"\n🔹 النتيجة {i} | شبه≈{sim:.3f} | المادة {meta.get('article')} | تكملة؟ {meta.get('is_continuation')}")
    print(doc[:400])



🔹 النتيجة 1 | شبه≈0.849 | المادة 639 | تكملة؟ False
شخص مقنع أو يحمل سلاحا ظاهرا أو خفيا ب .
6- إذا وقعت السرقة باستعمال العنف على الأشخاص.

المادة 639- معدلة وفقا للقانون487 تاريخ8/12/ 1995
يعاقب بالأشغال الشاقة من ثلاث سنوات إلى سبع سنوات كل من يرتكب السرقة في إحدى الحالات الآتية:
1- و الكسر في الأماكن المقفلة المصانة بالجدران، مأهولة أم غير مأهولة، أو يتسلقها في بواسطة الخلع أ
الداخل أو الخارج أو باستعمال المفاتيح المصنعة أو أي أداة أخرى أو بعد 

🔹 النتيجة 2 | شبه≈0.836 | المادة 635 | تكملة؟ False
ل شخص اشترك باللعب في الأماكن المذكورة أعلاه أو فوجيء فيها أثناء اللعب يعاقب بغرامة من خمسين
ألف إلى أربعمائة ألف ليرة.

-الجرائم التي تقع على الأموال الباب الحادي عشر

-في أخذ مال الغير الفصل الأول

النبذة
1- في السرقة

المادة 635- معدلة وفقا للمرسوم الاشتراعي112 تاريخ16/9/ 1983
- نقول خفية أو عنوة بقصد التملك تنزل الطاقات المحرزة منزلة الأشياء السرقة هي أخذ مال الغير الم
المنقولة في تطبيق النصوص

🔹 النتيجة 3 | شبه≈0.835 | المادة 637 | تكملة؟ False
لسرقة في إحدى الحالات التالية:
1- في ال

In [5]:
def build_context_block_with_meta(hits):
    lines = []
    for sim, meta, doc in hits:
        art = meta.get("article")
        tag = f"المادة {art}" if art else "مادة غير معروفة"
        snippet = doc.strip()
        if len(snippet) > 1200:
            snippet = snippet[:1200] + "…"
        lines.append(f"[{tag}] {snippet}")
    return "\n\n".join(lines)

ctx = build_context_block_with_meta(hits)
print(ctx[:1000])


[المادة 639] شخص مقنع أو يحمل سلاحا ظاهرا أو خفيا ب .
6- إذا وقعت السرقة باستعمال العنف على الأشخاص.

المادة 639- معدلة وفقا للقانون487 تاريخ8/12/ 1995
يعاقب بالأشغال الشاقة من ثلاث سنوات إلى سبع سنوات كل من يرتكب السرقة في إحدى الحالات الآتية:
1- و الكسر في الأماكن المقفلة المصانة بالجدران، مأهولة أم غير مأهولة، أو يتسلقها في بواسطة الخلع أ
الداخل أو الخارج أو باستعمال المفاتيح المصنعة أو أي أداة أخرى أو بعد الدخول إليها بالحيلة أو بانتحال
صفة موظف أو بالتذرع بمهمة رسمية أو بأي طريقة أخرى غير مألوفة.
2- قوع أعمال شغب أو ثورة أو حرب أو حريق أو اضطراب أمني أو غرق سفينة أو أي نائبة في خلال و
أخرى.
3- بالتهديد بالسلاح أن لتهيئة الجريمة أم لتسهيلها أم للاستيلاء على المسروق أم لتأمين الهرب.

المادة640- معدلة وفقا للقانون487 تاريخ8/12/ 1995
- ذكورة في المادة السابقة عنف على الأشخاص أن لتهيئة الجريمة أم لتسهي

[المادة 635] ل شخص اشترك باللعب في الأماكن المذكورة أعلاه أو فوجيء فيها أثناء اللعب يعاقب بغرامة من خمسين
ألف إلى أربعمائة ألف ليرة.

-الجرائم التي تقع على الأموال الباب الحادي عشر

-في